<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [3]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [4]:
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [6]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [7]:
tokens = []
for i in df['reviews.text']:
    tokens.append(tokenize(i))

df['tokens'] = tokens
df.head()
    

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]"


In [8]:
id2word = corpora.Dictionary(df['tokens'])

In [9]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [10]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [11]:
corpus[345][:10]

[(17, 1), (18, 1), (21, 1), (34, 1)]

In [12]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [13]:
lda.print_topics()

[(0,
  '0.065*"great" + 0.052*"tablet" + 0.052*"kids" + 0.024*"friendly" + 0.020*"battery" + 0.018*"old" + 0.017*"year" + 0.017*"love" + 0.016*"user" + 0.014*"life"'),
 (1,
  '0.064*"books" + 0.059*"read" + 0.025*"kindle" + 0.022*"reading" + 0.019*"easy" + 0.019*"games" + 0.018*"love" + 0.014*"light" + 0.013*"device" + 0.012*"book"'),
 (2,
  '0.048*"tablet" + 0.029*"amazon" + 0.028*"great" + 0.018*"best" + 0.015*"buy" + 0.014*"tablets" + 0.013*"use" + 0.012*"apps" + 0.010*"bought" + 0.010*"work"'),
 (3,
  '0.029*"tablet" + 0.029*"amazon" + 0.016*"device" + 0.016*"use" + 0.012*"apps" + 0.012*"great" + 0.012*"music" + 0.011*"prime" + 0.010*"free" + 0.009*"alexa"'),
 (4,
  '0.054*"great" + 0.043*"tablet" + 0.025*"good" + 0.023*"easy" + 0.018*"parental" + 0.018*"use" + 0.016*"controls" + 0.015*"excellent" + 0.014*"kids" + 0.010*"features"'),
 (5,
  '0.043*"tablet" + 0.041*"screen" + 0.028*"nice" + 0.022*"good" + 0.018*"little" + 0.016*"great" + 0.014*"size" + 0.013*"price" + 0.012*"apps" +

In [14]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [15]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\alfar\anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.251387 -0.011600       1        1  22.183573
11     0.161516  0.078698       2        1   8.490597
13     0.072571 -0.066977       3        1   7.711013
3     -0.014782 -0.128855       4        1   6.466817
7     -0.040631  0.189650       5        1   5.946299
10    -0.053536 -0.003246       6        1   5.798196
5     -0.012659 -0.070007       7        1   5.720197
2     -0.033407 -0.026281       8        1   5.506734
6      0.056253  0.022459       9        1   5.157382
1      0.122336 -0.127423      10        1   5.040336
8     -0.195013  0.033886      11        1   4.875397
0      0.067812  0.085657      12        1   4.521442
4      0.076274  0.088135      13        1   4.398423
9     -0.002822 -0.090296      14        1   4.186844
12     0.047476  0.026201      15        1   3.996752, topic_info=           Term         Freq        Total Category  logprob  loglift
18    batteries  7235.000000  7235.000000  Default  30.0000  30.0000
21        great  8895.000000  8895.000000  Default  29.0000  29.0000
34         good  5187.000000  5187.000000  Default  28.0000  28.0000
2074     kindle  3261.000000  3261.000000  Default  27.0000  27.0000
2431     tablet  7319.000000  7319.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
40     purchase    71.218168   951.161978  Topic15  -5.1502   0.6278
651       games    71.464191  1833.675118  Topic15  -5.1468  -0.0252
813       prime    61.605849   542.307021  Topic15  -5.2952   1.0446
105        year    63.161453  1861.760999  Topic15  -5.2703  -0.1639
234      device    61.712651  1109.797289  Topic15  -5.2935   0.3302

[1195 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
56        1  0.887387    aa
56        4  0.002224    aa
56        9  0.002224    aa
56       11  0.104529    aa
57        1  0.779191   aaa
...     ...       ...   ...
2983      4  0.018452   äôt
2983      8  0.110710   äôt
2983      9  0.092258   äôt
2983     15  0.092258   äôt
3579      3  0.913027  äôve

[4626 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 12, 14, 4, 8, 11, 6, 3, 7, 2, 9, 1, 5, 10, 13])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling